In [1]:
sc = SparkContext.getOrCreate()

def atualizar(urls, rank):
    for i in urls: yield(i, rank/len(urls))

paginas = "paginaA","paginaB"
paginas0 = "paginaB","paginaC"
paginas1 = "paginaC","paginaA"
paginas2 = "paginaA","paginaC"

links = (sc
        .parallelize([paginas,paginas0,paginas1,paginas2])
        .map(lambda x: [x[0],x[1]])
        .cache()
        )
for i in links.collect(): print i
    
grupos = (links
         .groupByKey()
         .cache()
         )
for i in grupos.collect(): print i
    
ranks = (grupos
        .map(lambda x: (x[0], 1.0))
        )
for i in ranks.collect() : print i

    
for x in range(2): gruposRankeados = grupos.join(ranks)
for i in gruposRankeados.collect() : print i
    
rankAtual = (gruposRankeados
            .flatMap(lambda x: atualizar(x[1][0], x[1][1]))
            )
for i in rankAtual.collect() : print i
    
rankFinal = (rankAtual
            .reduceByKey(lambda x,y: x+y)
            .mapValues(lambda d: d * 0.85 + 0.15)
            )
for i in rankFinal.collect() : print i
    
for (pagina, rank) in rankFinal.sortBy(lambda x:-x[1]).take(3):
    print("\n\nA %s está com %s no ranking." % (pagina, rank))


['paginaA', 'paginaB']
['paginaB', 'paginaC']
['paginaC', 'paginaA']
['paginaA', 'paginaC']
('paginaA', <pyspark.resultiterable.ResultIterable object at 0x0000000008C10A90>)
('paginaC', <pyspark.resultiterable.ResultIterable object at 0x0000000008C10BA8>)
('paginaB', <pyspark.resultiterable.ResultIterable object at 0x0000000008C10AC8>)
('paginaA', 1.0)
('paginaC', 1.0)
('paginaB', 1.0)
('paginaA', (<pyspark.resultiterable.ResultIterable object at 0x0000000008C308D0>, 1.0))
('paginaC', (<pyspark.resultiterable.ResultIterable object at 0x0000000008C30908>, 1.0))
('paginaB', (<pyspark.resultiterable.ResultIterable object at 0x0000000008C30630>, 1.0))
('paginaB', 0.5)
('paginaC', 0.5)
('paginaA', 1.0)
('paginaC', 1.0)
('paginaA', 1.0)
('paginaC', 1.4249999999999998)
('paginaB', 0.575)


A paginaC está com 1.425 no ranking.


A paginaA está com 1.0 no ranking.


A paginaB está com 0.575 no ranking.
